# ResNet50 for Kidney Disease Classification

## import functions and libraries

In [ ]:
import torchvision
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.models import resnet50, ResNet50_Weights
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import wandb
import opendatasets as od
from pathlib import Path
from collections import Counter
import splitfolders
from sklearn.metrics import precision_score,recall_score

## path to data

In [ ]:
dataset="https://www.kaggle.com/datasets/nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone"

od.download(dataset)

In [ ]:
data_path='ct-kidney-dataset-normal-cyst-tumor-and-stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'
splitfolders.ratio(data_path, output='dataset',seed=831, ratio=(.8, 0.1,0.1))

In [ ]:
trans=transforms.Compose([transforms.Resize((224,224)),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
                         
                         ])

In [ ]:
train_dataset=datasets.ImageFolder(root="dataset/train",transform=trans)
valid_dataset=datasets.ImageFolder(root="dataset/val",transform=trans)
test_dataset=datasets.ImageFolder(root="dataset/test",transform=trans)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
valid_loader=DataLoader(valid_dataset,batch_size=32,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:

classifier = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
classifier.fc=nn.Linear(2048,4)

In [ ]:
classes=(train_dataset.classes)
train_dataset=datasets.ImageFolder(root="dataset/train",transform=trans)
fig = plt.figure(figsize=(30, 30))

images,labels=next(iter(train_loader))
img = images / 2 + 0.5 
grid_img = torchvision.utils.make_grid(img)
plt.imshow(grid_img.permute(1, 2, 0),cmap='gray')

plt.title(' '.join('%20s, ' % classes[labels[j]] for j in range(len(labels))))


In [ ]:
import pandas as pd
data=pd.read_csv('ct-kidney-dataset-normal-cyst-tumor-and-stone\kidneyData.csv')



In [ ]:
category=np.unique(data['Class'])
category
id,count=np.unique(data['Class'],return_counts=True)
id,count

In [ ]:
median=np.median(count)
class_weight=[]
for i in range(len(category)):
    print(f'{category[i]} : {median/count[i]}')
    class_weight.append(median/count[i])
class_weight=torch.FloatTensor(class_weight)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device


## train and evaluate model

In [ ]:

def train_eval(run,optimizer,device,classifier,epochs,class_weight,train_loader,valid_loader,test_loader):
   
    wandb.watch(classifier, log='all')
    classifier.train()
    classifier=classifier.to(device)
    class_weight=class_weight.to(device)
    
    for epoch in tqdm(range(epochs)):
        train_total_loss=0.0
        num_batch=0
        
        train_total_accuracy=0.0
        train_total=0
        for batch_size,(image,labels) in tqdm(enumerate(train_loader)):
           
            image=image.to(device)
            labels=labels.to(device)
            output=classifier(image)
            
            optimizer.zero_grad()
            loss=F.cross_entropy(output,labels,weight=class_weight)
            loss.backward()
            predicted=torch.argmax(output,dim=-1)
            
            optimizer.step()
            num_batch+=1
            train_total_loss+=loss.item()
            train_total+=len(labels)
           
            train_total_accuracy+=(predicted == labels).sum().item() 
            
        
        wandb.log({
        'average_train_loss': train_total_loss/num_batch,
        'average_train_accuracy' :train_total_accuracy/train_total    
            })
        classifier.eval()
        val_total_loss=0.0
        val_total_accuracy=0.0
        val_total_precision_considering_class=0.0
        val_total_recall_considering_class=0.0
        val_total=0
        num_batch=0
        
        with torch.no_grad():
            for image,labels in tqdm(valid_loader):
                
                image=image.to(device)
                labels=labels.to(device)
                output=classifier(image)
                loss=F.cross_entropy(output,labels,weight=class_weight)
                
                _,predicted=torch.max(output,1)
                num_batch+=1
                val_total_loss+=loss.item()
                val_total_accuracy+=(predicted == labels).sum().item() 
                val_total_precision_considering_class+=precision_score(labels.cpu().numpy(),predicted.cpu().numpy(),average='weighted',zero_division=1)
                val_total_recall_considering_class+=recall_score(labels.cpu().numpy(),predicted.cpu().numpy(),average='weighted',zero_division=1)
               
                val_total+=len(labels)
           
            
           
            
            wandb.log({
            'average_val_loss': val_total_loss/num_batch,
            'average_val_accuracy' :val_total_accuracy/val_total,    
            'average_val_precsion' :val_total_precision_considering_class/num_batch,    
            'average_val_recall' :val_total_recall_considering_class/num_batch
       
        })
    with torch.no_grad():
        classifier.eval()
        test_total_loss=0.0
        test_total_accuracy=0.0
        test_total_precision_considering_class=0.0
        test_total_recall_considering_class=0.0
        test_total=0
        num_batch=0        
        for image,labels in tqdm(test_loader):
            
            image=image.to(device)
            labels=labels.to(device)
            output=classifier(image)
            loss=F.cross_entropy(output,labels,weight=class_weight)
            
            _,predicted=torch.max(output,1)
            num_batch+=1
            test_total_loss+=loss.item()
            test_total_accuracy+=(predicted == labels).sum().item() 
            test_total_precision_considering_class+=precision_score(labels.cpu().numpy(),predicted.cpu().numpy(),average='weighted',zero_division=1)
            test_total_recall_considering_class+=recall_score(labels.cpu().numpy(),predicted.cpu().numpy(),average='weighted',zero_division=1)
           
            test_total+=len(labels)
                
    print(f'average_test_loss: {test_total_loss/num_batch}\naverage_test_accuracy :{test_total_accuracy/test_total}\naverage_test_precsion :{test_total_precision_considering_class/num_batch}\naverage_test_recall :{test_total_recall_considering_class/num_batch}')
    torch.save(classifier.state_dict(),'kidney_model.pt')
    artifact=wandb.Artifact('model',type='model')
    artifact.add_file('kidney_model.pt')
    run.log_artifact(artifact)  
    



In [ ]:

sweep_config={
'name' : 'parameter_tuning',
'method': 'grid',
'metric': {'name': 'val_loss','goal' :'minimize'},
'parameters':{
    'learning_rate' :
     {'values': [10**-5,10**-3]},
    'epochs': 
    {'values':[1,5,10]}
    
    },
'early_terminate':{
'type': 'hyperband',
'min_iter': 3
}
}




## run sweep and look at configuration on wandb dashboard 

In [ ]:
def run_sweep(config=None):
    
    
    run=wandb.init(config=config)
    wandb_config=wandb.config
    loss=nn.CrossEntropyLoss(weight=class_weight)
    train_dataset=datasets.ImageFolder(root="dataset/train",transform=trans)
    valid_dataset=datasets.ImageFolder(root="dataset/val",transform=trans)
    test_dataset=datasets.ImageFolder(root="dataset/test",transform=trans)
    train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
    valid_loader=DataLoader(valid_dataset,batch_size=32,shuffle=False)
    test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)
    optimizer=torch.optim.Adam(classifier.parameters(),lr=wandb_config.learning_rate)
    train_eval(run,optimizer,device,classifier,wandb_config.epochs,class_weight,train_loader,valid_loader,test_loader)


In [ ]:
sweep_id=wandb.sweep(sweep_config,project='kidney_classification')
wandb.agent(sweep_id,run_sweep)
wandb.finish()